In [1]:
import pandas as pd
import requests
import time
import random
import html
import logging
from tqdm import tqdm


In [2]:
# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
import pandas as pd
import re
import time
import requests
from tqdm import tqdm

def limpiar_nombre(nombre):
    if pd.isna(nombre) or not nombre.strip():
        return None
    nombre = nombre.replace("\\", "").replace('"', '').replace("'", "")
    nombre = nombre.replace(",", "").replace("/", " ").replace("&", "and")
    return nombre.strip()

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])
nombres_limpios = [limpiar_nombre(nombre) for nombre in df["raw"]]
artistas_unicos = sorted(set([nombre for nombre in nombres_limpios if nombre]))

print(f"✅ Total artistas únicos: {len(artistas_unicos)}")

# =======================
# 2. CONSULTA SPARQL SIN "death"
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
HEADERS = {
    "Accept": "application/sparql-results+json",
    "User-Agent": "Workshop/1.0 (tucorreo@example.com)"  # <-- Cámbialo por el tuyo
}

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?countryLabel ?awardLabel ?genderLabel (COUNT(?album) AS ?albumCount) WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      OPTIONAL {{ ?artist wdt:P166 ?award. }}
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}

      OPTIONAL {{
        ?album wdt:P31 wd:Q482994.
        ?album wdt:P175 ?artist.
      }}

      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    GROUP BY ?artistLabel ?countryLabel ?awardLabel ?genderLabel
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 80
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", "No awards"),
                        "gender": row.get("genderLabel", {}).get("value", "Unknown"),
                        "album_count": row.get("albumCount", {}).get("value", "0")
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(0.8)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
columnas_ordenadas = ["artist", "country", "award", "gender", "album_count"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)


✅ Total artistas únicos: 31437
🚀 Consultando Wikidata...


🔎 Batches SPARQL: 31440it [10:59, 47.69it/s]                           


In [3]:
# Calcular el punto de división (por la mitad en este caso)
halfway = len(df_result) // 2

# Guardar la primera parte del DataFrame
df_result.iloc[:halfway].to_csv("../data/api_data_part1.csv", index=False)
print("✅ Archivo 'api_data_part1.csv' creado correctamente.")

# Guardar la segunda parte del DataFrame
df_result.iloc[halfway:].to_csv("../data/api_data_part2.csv", index=False)
print("✅ Archivo 'api_data_part2.csv' creado correctamente.")

✅ Archivo 'api_data_part1.csv' creado correctamente.
✅ Archivo 'api_data_part2.csv' creado correctamente.
